In [1]:
import pandas as pd
import statistics as st

In [2]:
resNuevos = pd.read_csv('resultados_experimentos_modelo_nuevo_40clientes.csv')
resViejos = pd.read_csv('resultados_experimentos_modelo_viejo_40clientes.csv')

In [3]:
tabla = pd.merge(resNuevos, resViejos, on = 'archivo')
tabla.head()

,archivo,status_x,valor_objetivo_x,tiempo_segundos_x,status_y,valor_objetivo_y,tiempo_segundos_y
0,instancia_0_40_clientes.txt,integer optimal solution,50004.0,8.737428,"integer optimal, tolerance",50004.0,1.539896
1,instancia_1_40_clientes.txt,"integer optimal, tolerance",37539.0,125.996328,"integer optimal, tolerance",55882.0,2.000715
2,instancia_2_40_clientes.txt,"integer optimal, tolerance",48623.0,1.173780,integer optimal solution,48623.0,0.997455
3,instancia_3_40_clientes.txt,"integer optimal, tolerance",53895.0,0.699697,"integer optimal, tolerance",53895.0,0.904926
4,instancia_4_40_clientes.txt,"integer optimal, tolerance",48805.0,13.491060,"integer optimal, tolerance",48805.0,5.639760


In [12]:
tpViejo = tabla.tiempo_segundos_y.mean()
tpExtra = tabla.tiempo_segundos_x.mean()

print("Tiempo promedio en resolver una instancia: ")
print(f"Modelo viejo: {tpViejo:.2f}s")
print(f"Modelo nuevo con restricciones extra: {tpExtra:.2f}s")
print(f"En promedio el modelo nuevo con restricciones extra tarda {(tpExtra/tpViejo -1)*100:.2f}% mas en resolver una instancia de 40 clientes que el modelo viejo")

Tiempo promedio en resolver una instancia: 
Modelo viejo: 12.63s
Modelo nuevo con restricciones extra: 365.76s
En promedio el modelo nuevo con restricciones extra tarda 2795.06% mas en resolver una instancia de 40 clientes que el modelo viejo


In [11]:
objE = tabla.valor_objetivo_x.mean()
objV = tabla.valor_objetivo_y.mean()
print(f"promedio valor objetivo -> modelo viejo: {objV}, modelo restricciones extra: {objE}")

print(f"El modelo nuevo con restricciones extra reduce el costo promedio en {100 -(objE*100)/objV:.2f} %")

promedio valor objetivo -> modelo viejo: 52003.66666666667, modelo restricciones extra: 48144.78571428588
El modelo nuevo con restricciones extra reduce el costo promedio en 7.42 %
